# Intro
* familiar bit: few shot, chain of reasoning style prompt
* works with off the shelf models, no fine tuning, you can use this method with LLMs you get over api calls.
* TODO: screenshot of cot prompt

# Algorithm
* screen shot of algo demo from paper
* picture of Jon's diagram
       

# code
babbage results

In [93]:
%%capture
! pip install openai

In [3]:
from openai import Completion
import openai

In [96]:
openai.api_key_path = '/root/.openai/key'

In [97]:
few_shot_prompt='''Please answer the given question by writing out the chain of reasoning, and then saying the answer. End your answer in the form 'The answer is <answer>.'

Examples:

Q: John has 2 eggs. He gets 3 more from the store. How many eggs does he end up with?
A: Starting with 2 he gets 3 more. 2 + 3 = 5. The answer is 5.

Q: Bill makes $12 an hour. He works for 4 hours Monday and 5 hours Tuesday. How much money does he make these two days?
A: Monday he made 4 * $12 = $48, and Tuesday he made 5 * $12 = $60. In total he made $48 + $60 = $108. The answer is $108.

Q: There are 4 birds in a tree. 3 more fly in to join them. How many birds are there total?
A: First there are 4, and 3 more makes 4 + 3 = 7. The answer is 7.

Q: Last year Dale bought a book for $18.30, which was 25% off the sticker price. What was the sticker price?
A: '''

In [ ]:
single = Completion.create(prompt=few_shot_prompt,
                      model="davinci-002", max_tokens=75, n=1, temperature=0.0)
single['choices'][0]['text']

In [88]:
completions = Completion.create(prompt=few_shot_prompt,
                      model="davinci-002", temperature=.5,  max_tokens=75, n=50, stop="Q: ")

In [89]:
for response in completions['choices'][:3]:
    print('=====\n' + response['text'])



=====
100% of the sticker price is $18.30. 100% / 25% = 4. The answer is $18.30 * 4 = $73.20.


=====
25% of $18.30 is $4.57. The sticker price is $18.30 + $4.57 = $22.87. The answer is $22.87.


=====
18.30 / 0.75 = 24.4, so the sticker price was $24.40.




In [90]:
import re
pattern = re.compile(r'answer is ([^\s]+)')
answers = []
for response in completions['choices']:
    matches = pattern.findall(response['text'])
    if len(matches) > 0:
        answers.append(matches[0])
answers

['$18.30',
 '$22.87.',
 '$24.40.',
 '$13.73.',
 '$24.40.',
 '$24.40.',
 '$32.05.',
 '$24.40.',
 '$24.40.',
 '$24.40.',
 '$24.40.',
 '$22.875.',
 '22.875.',
 '$13.725.',
 '$24.40.',
 '$24.40.',
 '$24.40.',
 '$24.40.',
 '$22.87.',
 '$22.88.',
 '$4.57.',
 '$24.40.',
 '$24.40.',
 '$24.40.',
 '$13.73.',
 '$24.40.',
 '$13.73.',
 '$22.88.',
 '$23.12.',
 '$22.87.',
 '$24.40.']

In [91]:
from collections import Counter
ctr = Counter(answers)
ctr.most_common(5)

[('$24.40.', 16),
 ('$22.87.', 3),
 ('$13.73.', 3),
 ('$22.88.', 2),
 ('$18.30', 1)]

# Results
## parameters
40 samples, temperature sampling t=.5, topk=40, but results compatible with many sampling strategies
## specific result improvement example
davinci-002 / GSM8k, 60.1 -> 78, + 17.9% difference
screenshot of the gsm8k question

## Screenshot of the greedy vs. sampled paths for palm-540b
screenshot of row example from appendix, palm also saw 17.9% improvement

## table of results
lots of info

# Compare to other approaches 
* multimodel ensemble - only 3, unsure exactly how algo worked for this, but did worse
* ensemble of prompts - confusing
* beam search (used by itself)

# Effects of parameters
* num reasoning paths
  * perf saturation after 10
* temperature, top k, top p
* sampling method - beam search works, but has less diversity than other methods

# Observations
* bigger effect with higher model size
* consistency % is confidence indicator
  * show graph
* trade 1 time compute at train for permanent 10x or more compute cost increase